In [10]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import warnings
from datetime import datetime
warnings.filterwarnings('ignore')
import random
from sklearn.model_selection import train_test_split

In [11]:
con = sqlite3.connect(r"../laliga.sqlite")
# Change date format (2029 -> 1929)
df = pd.read_sql_query("SELECT * FROM Matches", con)

# Data formatting:
# We split the "date" column into the corresponding columns, and specify the correct formatting.
# This way we avoid the year 29 or higher to be mistaken by 2029 (error with to_datetime function).
df[["month", "day", "year"]] = df["date"].str.split("/", 2, expand = True)
df["year"] = df["year"].apply(lambda x: f"19{x}" if int(x) >= 23 else f"20{x}")
df['date'] = pd.to_datetime(df[['day', 'month', 'year']]).dt.strftime('%d/%m/%Y') 

# We also change the formatting of the time to be in 24h format
df['time'] = df['time'].apply(lambda x: datetime.strftime(datetime.strptime(x, "%I:%M %p"), "%H:%M") if type(x) == str else x)

df[["home_goals", "away_goals"]] = df["score"].str.split(":", expand=True).astype(float)
df2=df.dropna()
df2["goal diff"] = (df2["home_goals"]  - df2["away_goals"])
df2["results"] = np.where(df2["goal diff"] > 0, "1", np.where(df2["goal diff"] < 0, "2", "X"))
df2.head()



,season,division,matchday,date,time,home_team,away_team,score,month,day,year,home_goals,away_goals,goal diff,results
16024,1995-1996,1,1,03/09/1995,19:30,CP Mérida,Real Betis,1:1,9,3,1995,1.0,1.0,0.0,X
17318,1998-1999,1,1,29/08/1998,21:00,Alavés,Real Betis,0:0,8,29,1998,0.0,0.0,0.0,X
17319,1998-1999,1,1,29/08/1998,22:00,Valencia,Atlético Madrid,1:0,8,29,1998,1.0,0.0,1.0,1
17320,1998-1999,1,1,30/08/1998,19:00,Celta de Vigo,Dep. La Coruña,0:0,8,30,1998,0.0,0.0,0.0,X
17321,1998-1999,1,1,30/08/1998,19:00,Real Sociedad,Real Oviedo,3:3,8,30,1998,3.0,3.0,0.0,X


Features:
1. Home (1) vs. Visitor (0)
2. Sum goals
3. Sum received goals (conceed)
4. Diff 2&3
5. Diff goles entre equipos
6. Season percentage (0-1). A qué altura de la temporada estamos (0 inicio, 1 final). 
7. Time of game (0-24).
8. Division
9. Number of points during the season.
10. Absolute rankings.

In [13]:
#1
df2['home_win'] = df['home_goals'] > df['away_goals']
#2
df2['total_goals_sum'] = 0
pd.crosstab(df2['home_team'], df2['total_goals_sum'], df2['home_goals'], aggfunc=sum)

total_goals_sum,0
home_team,
Alavés,388.0
Albacete,371.0
Alcorcón,290.0
Algeciras CF,16.0
Alicante CF,21.0
...,...
UE Lleida,22.0
Valencia,702.0
Villarreal,660.0


In [ ]:
y = df2.results
x = df2[["home_team", "away_team"]]
#x = data.drop('temp',axis=1)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

In [ ]:
import pickle


class QuinielaModel:

    def train(self, train_data):
        # Do something here to train the model
        pass

    def predict(self, predict_data):
        # Do something here to predict
        return ["X" for _ in range(len(predict_data))]